In [1]:

import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [5]:
# 1. Tải dữ liệu đã được trích xuất

FEATURES_DIR = "../features_advanced" 

# Tải tập train
x_train = np.load(f"{FEATURES_DIR}/train/features.npy")
y_train = np.load(f"{FEATURES_DIR}/train/labels.npy")

# Tải tập validation (để kiểm tra trong quá trình huấn luyện)
x_val = np.load(f"{FEATURES_DIR}/val/features.npy")
y_val = np.load(f"{FEATURES_DIR}/val/labels.npy")

# Tải tập test (chỉ dùng để đánh giá cuối cùng)
x_test = np.load(f"{FEATURES_DIR}/test/features.npy")
y_test = np.load(f"{FEATURES_DIR}/test/labels.npy")

print(f"Kích thước tập train: X={x_train.shape}, y={y_train.shape}")
print(f"Kích thước tập val:   X={x_val.shape}, y={y_val.shape}")
print(f"Kích thước tập test:  X={x_test.shape}, y={y_test.shape}")
print("-" * 30)

Kích thước tập train: X=(921, 1883), y=(921,)
Kích thước tập val:   X=(231, 1883), y=(231,)
Kích thước tập test:  X=(288, 1883), y=(288,)
------------------------------


In [ ]:
# 2. Huấn luyện mô hình XGBoost
print("Bắt đầu huấn luyện mô hình XGBoost...")

# Tinh chỉnh các tham số để có hiệu năng tốt hơn
xgb_model = XGBClassifier(
    objective='multi:softmax', # Xác định bài toán phân loại nhiều lớp
    num_class=len(np.unique(y_train)), # Số lượng lớp cảm xúc
    n_estimators=1000,         # Tăng số lượng cây
    learning_rate=0.05,
    max_depth=8,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,   # Tắt warning
    eval_metric='mlogloss',    # Metric để theo dõi
    random_state=42,
    n_jobs=-1,                  # Sử dụng tất cả các CPU cores
    early_stopping_rounds=50
)
# Sử dụng tập validation để theo dõi và dừng sớm nếu không cải thiện
# Điều này giúp chống overfitting hiệu quả
xgb_model.fit(
    x_train, y_train,
    eval_set=[(x_val, y_val)],
    # Dừng nếu val_loss không cải thiện trong 50 vòng
    verbose=50 # In ra kết quả mỗi 50 vòng
)
print("\n✅ Huấn luyện hoàn tất!")
print("-" * 30)

Bắt đầu huấn luyện mô hình XGBoost...


C:\Users\FPT SHOP\anaconda3\Lib\site-packages\xgboost\callback.py:386: UserWarning: [10:07:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  self.starting_round = model.num_boosted_rounds()


[0]	validation_0-mlogloss:2.03419
[50]	validation_0-mlogloss:1.36146


In [ ]:
# 3. Đánh giá mô hình trên tập Test

print("📈 Đánh giá mô hình trên tập TEST...")
y_pred = xgb_model.predict(x_test)

# Lấy mapping từ file preprocess gốc để hiển thị tên nhãn
EMOTION_MAP = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}
target_names = list(EMOTION_MAP.values())

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy trên tập Test: {accuracy * 100:.2f}%")

# Classification Report
print("\nBáo cáo chi tiết (Classification Report):")
print(classification_report(y_test, y_pred, target_names=target_names))

# Confusion Matrix
print("\nMa trận nhầm lẫn (Confusion Matrix):")
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix on Test Set')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()